# Top 50/75 Service Checker
* A 22.4 hackfest projects by some manager types who still like to play
* Iterated in PI 23.2 to add query pagination.

**Goal:**
* Dashboard the level of services available for our Top 50/75 datasets (limited to those discoverable via CMR)

**Strech Goals:**
* quantify (and visualize) the problem of “I have to do something different for different datasets every time I access/read/plot”
* if we have x services on x datasets - can we search/access/read/viz them all the same way across those different endpoints?

## Install prerequisite packages:

Note: `notebook` must be installed first within your Python environment to run a Jupyter notebook server.

In [ ]:
import sys

!{sys.executable} -m pip install pandas matplotlib seaborn requests

In [ ]:
%matplotlib inline
import json

import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns

## Top 50 and Stretch 75 collection short names:

GraphQL queries use collection concept IDs as some collections (GLDAS) have multiple results.

Collection specific notes:

* DAYMET_DAILY_V4_1840 has been updated to Daymet_Daily_V4R1_2129
* GLDAS_NOAH025_3H - multiple matches. Likely actual match is C1342986035-GES_DISC
* GLDAS_NOAH10_3H - multiple matches. Likely actual match is C1288937556-GES_DISC
* PODAAC-OSCAR-03D01 has been updated to OSCAR_L4_OC_third-deg
* PODAAC-GHMDA-2PJ19 has no matching data MODIS_A-JPL-L2P-v2019.0
* PODAAC-GHGMR-4FJ04 has no matching data MUR-JPL-L4-GLOB-v4.1
* ASTGTM_NC is available as metadata, but granules seem to be inaccessible (ACL related?)

In [ ]:
top_datasets = {
    "SENTINEL-1A_SLC": "C1214470488-ASF",
    "GPM_3IMERGHH": "C1598621093-GES_DISC",
    "NLDAS_FORA0125_H": "C2033151148-GES_DISC",
    "M2T1NXSLV": "C1276812863-GES_DISC",
    "GLDAS_NOAH025_3H": "C1342986035-GES_DISC",
    "GPM_3IMERGHHL": "C1598621095-GES_DISC",
    "GPM_3IMERGHHE": "C1598621094-GES_DISC",
    "NLDAS_NOAH0125_H": "C2069246977-GES_DISC",
    "M2I3NPASM": "C1276812879-GES_DISC",
    "M2I1NXASM": "C1276812820-GES_DISC",
    "GPM_3IMERGDF": "C1598621096-GES_DISC",
    "M2T1NXFLX": "C1276812838-GES_DISC",
    "M2T1NXRAD": "C1276812851-GES_DISC",
    "NLDAS_MOS0125_H": "C2069247178-GES_DISC",
    "GPM_MERGIR": "C1432254058-GES_DISC",
    "NLDAS_VIC0125_H": "C2069247359-GES_DISC",
    "GLDAS_NOAH10_3H": "C1288937556-GES_DISC",
    "S5P_L2__NO2___": "C1442068511-GES_DISC",
    "S5P_L2__CO____": "C1442068493-GES_DISC",
    "S5P_L2__CH4___": "C1442068505-GES_DISC",
    "S5P_L2__SO2___": "C1442068508-GES_DISC",
    "MOD04_L2": "C1443533440-LAADS",
    "MYD04_L2": "C1443533683-LAADS",
    "VNP46A2": "C1898025206-LAADS",
    "MYD03": "C1379841358-LAADS",
    "MYD021KM": "C1379758607-LAADS",
    "MYD06_L2": "C1443536017-LAADS",
    "MYD35_L2": "C1443715587-LAADS",
    "MOD03": "C1379767668-LAADS",
    "MOD021KM": "C1378227407-LAADS",
    "MOD06_L2": "C1443535037-LAADS",
    "VNP46A1": "C1897815356-LAADS",
    "MOD07_L2": "C1443541366-LAADS",
    "MOD05_L2": "C1443531026-LAADS",
    "MOD02HKM": "C1378577630-LAADS",
    "MCD06COSP_D3_MODIS": "C1887589686-LAADS",
    "MYD04_3K": "C1443528505-LAADS",
    "MYD07_L2": "C1443544285-LAADS",
    "MOD04_3K": "C1443420430-LAADS",
    "MOD35_L2": "C1443561895-LAADS",
    "MYD05_L2": "C1443531357-LAADS",
    "MOD11A1": "C1748058432-LPCLOUD",
    "MYD11A1": "C1748046084-LPCLOUD",
    "MOD11A2": "C2269056084-LPCLOUD",
    "MYD11A2": "C2269057787-LPCLOUD",
    "MCD19A2": "C2324689816-LPCLOUD",
    "MYD09GA": "C2202498116-LPCLOUD",
    "MOD15A2H": "C2218777082-LPCLOUD",
    "MOD09GA": "C2202497474-LPCLOUD",
    "MCD43A4": "C2218719731-LPCLOUD",
    "MOD14": "C2271754179-LPCLOUD",
    "MCD15A2H": "C2222147000-LPCLOUD",
    "MYD14": "C2278858993-LPCLOUD",
    "MCD43A3": "C2278860820-LPCLOUD",
    "MOD13Q1": "C1748066515-LPCLOUD",
    "MYD13Q1": "C2307290656-LPCLOUD",
    "MOD13A3": "C2327962326-LPCLOUD",
    "MYD13A3": "C2327957988-LPCLOUD",
    "MYD09GQ": "C2343109950-LPCLOUD",
    "MCD15A3H": "C2343110937-LPCLOUD",
    "ASTGTM": "C1711961296-LPCLOUD",
    "ASTGTM_NC": "C2439422590-LPCLOUD",
    "MOD09GQ": "C2343115666-LPCLOUD",
    "MOD09A1": "C2343111356-LPCLOUD",
    "MOD16A2": "C2343113232-LPCLOUD",
    "MOD09Q1": "C2343112831-LPCLOUD",
    "MYD09Q1": "C2343114343-LPCLOUD",
    "MYD09A1": "C2343113743-LPCLOUD",
    "MOD11_L2": "C2343115255-LPCLOUD",
    "MYD11_L2": "C2343114808-LPCLOUD",
    "Daymet_Daily_V4R1_2129": "C2532426483-ORNL_CLOUD",
    "MODIS_A-JPL-L2P-v2019.0": "C1940473819-POCLOUD",
    "OSCAR_L4_OC_third-deg": "C2036880665-POCLOUD",
    "MUR-JPL-L4-GLOB-v4.1": "C1996881146-POCLOUD",
    "VIIRS_NPP-STAR-L3U-v2.80": "C2147485059-POCLOUD"
}

provider_list = {
    "ASF", "GES_DISC", "LAADS", "LPCLOUD", "POCLOUD", "GHRC_DAAC", "ORNL_CLOUD", "NSIDC_CPRD"
}

## Scoring helper functions:

The main entry point for the cell below is `score_all_collections`, which will iterate through all rows and calculate both an ideal and an actual score for each collection.

In [ ]:
def is_harmony_service_with_variable_subsetting(service) -> bool:
    """ Check the service is a Harmony service and can do
        variable subsetting.

        Assignment to `service_options` is because a service with no
        options has `'serviceOptions': None}`, which is different to
        just not having the key at all.

    """
    service_options = service.get('serviceOptions', {}) or {}
    return (service.get('type') == 'Harmony'
            and 'variableSubset' in service_options.get('subset', {}))


def collection_has_variable_subsetting(collection_row: pd.Series) -> bool:
    """ A function to check if the following criteria are true:

        * The collection has at least 1 UMM-Var record.
        * The collection is associated with at least 1 Harmony service that
          lists variable subsetting in the service options of its UMM-S record.

    """
    services_json = collection_row['services.items']
    return (services_json is not None
            and any(is_harmony_service_with_variable_subsetting(service) for service in services_json))


def collection_has_compatible_native_format(collection_row: pd.Series) -> bool:
    """ Ensure that the collection has native format information
        and that at least one of the entries contains the substring
        'DF' (formats will be converted to upper-case for comparison).

    """
    native_formats = collection_row['nativeDataFormats']
    return (native_formats is not None
            and any('DF' in native_format.upper() for native_format in native_formats))


def get_ideal_score(collection_row: pd.Series) -> int:
    """ Retrieve an ideal score, primarily based on the file format. (For now) """
    if collection_has_compatible_native_format(collection_row):
        ideal_score = 5
    else:
        ideal_score = 0

    return ideal_score


def get_actual_score(collection_row: pd.Series) -> int:
    """ Retrieve an actual score for a collection to assess how compliant
        it is with the Levels of Service.

        This is overly simplistic!

        Background on the has<...> fields is here:
        https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html

        Ideal score is 5, 1 point each for:

        * Spatial subsetting
        * Temporal subsetting
        * Variable subsetting
        * Reformatting
        * Other (e.g., reprojection)

    """
    actual_score = 0

    if collection_row['hasSpatialSubsetting'] == True:
        actual_score += 1

    if collection_row['hasTemporalSubsetting']== True:
        actual_score += 1

    if collection_row['hasFormats'] == True:
        actual_score += 1

    if collection_row['hasTransforms'] == True:
        actual_score += 1

    if collection_has_variable_subsetting(collection_row):
        actual_score += 1

    return actual_score


def score_all_collections(collections_df: pd.DataFrame):
    """ Iterate through all rows and assign an ideal and an actual score. """
    collections_df['idealScore'] = [get_ideal_score(row[1]) for row in collections_df.iterrows()]
    collections_df['actualScore'] = [get_actual_score(row[1]) for row in collections_df.iterrows()]

## Creating a pandas compatible column for data formats:

The functions below extract the first listed native data format from the `nativeDataFormats` column of each collection. The `nativeDataFormats` column is a list, which does not allow for simple grouping and aggregation.

In [ ]:
def get_first_native_data_format(collection_row: pd.Series):
    """ We can't use pandas well with the native data formats since they are a list,
        so this yanks them out into their own columns.  Not perfect, but better than nothing.

    """
    native_formats = collection_row['nativeDataFormats']

    if native_formats is not None and len(native_formats) > 0:
        native_format = native_formats[0]
    else:
        native_format = None

    return native_format


def extract_all_first_native_data_formats(collections_df: pd.DataFrame):
    """ We can't use pandas well with the native data formats since they are a list,
        so this yanks them out into their own columns.  Not perfect, but better than nothing.

        Current version only grabs the first native version listed.

    """
    collections_df['nativeDataFormat1'] = [get_first_native_data_format(row[1])
                                            for row in collections_df.iterrows()]

## Performing generic collection-based queries

The cell below will perform a request against [CMR graph](https://graphql.earthdata.nasa.gov/api), using the specified query parameters. To avoid timeouts, basic pagination is implemented, retrieving only 100 results at a time.

In [ ]:
def query_cmr_graph_for_collections(query_parameters):
    """ Perform a query against CMR graph (https://graphql.earthdata.nasa.gov/api)
        to retrieve information for a given set of query parameters.

        Limit is set to 100 results to avoid timeouts. In particular when retrieving
        all POCLOUD collections.

    """
    graphql_rooturl = "https://graphql.earthdata.nasa.gov/api"
    query_parameters['limit'] = 100

    query_string = """
    query Collections($params: CollectionsInput){
      collections(params: $params) {
        count
        cursor
        items {
          provider
          shortName
          conceptId
          nativeDataFormats
          processingLevel
          hasFormats
          hasSpatialSubsetting
          hasTemporalSubsetting
          hasTransforms
          services {
            count
            items {
              type
              serviceOptions
            }
          }
          granules {
            count
          }
          variables {
            count
          }
        }
      }
    }
    """

    request_json = {'operationName': 'Collections',
                    'query': query_string,
                    'variables': {'params': query_parameters}}

    # Initialise an empty pandas DataFrame, to which results will be appended:
    all_the_data_df = pd.DataFrame()

    # Set collection count to a non-zero value to trigger first request:
    collection_count = 1

    # Add an error count and limit, to prevent infinite loops from repeated errors:
    error_count = 0
    max_errors = 3

    # Perform paginated request - this helps in particular with POCLOUD holdings
    # Requests will continue until the DataFrame is the same length as the
    # number of results in CMR graph.
    while error_count < max_errors and len(all_the_data_df) < collection_count:
        cmr_graph_response = requests.post(url=graphql_rooturl, json=request_json)

        if cmr_graph_response.ok:
            json_data = cmr_graph_response.json()

            # Update collection_count to actual value:
            collection_count = json_data['data']['collections']['count']

            # Concatenate new results with existing dataframe:
            df_data = json_data['data']['collections']['items']
            new_df = pd.json_normalize(df_data)
            all_the_data_df = pd.concat([all_the_data_df, new_df], ignore_index=True)

            # Update the cursor for the potential next request:
            request_json['variables']['params']['cursor'] = json_data['data']['collections']['cursor']
        else:
            error_count += 1
            print(f'Error with {collection_short_name}')
            print(f'response status code: f{response.status_code}')
            print(f'response : {response.content}')

    return all_the_data_df

# Retrieve Top 50/75 datasets:

* Perform a CMR graph query
* Extract the first listed native data format for all collections.
* Score level of service for all collections.

In [ ]:
topdata_df = query_cmr_graph_for_collections(
    {'cloudHosted': True, 'conceptId': list(top_datasets.values())}
)
extract_all_first_native_data_formats(topdata_df)
score_all_collections(topdata_df)
display(topdata_df)

# Visually display level of service compliance for top 50/75 collections:

In [ ]:
actual_score = topdata_df.groupby(['processingLevel.id'])['actualScore'].sum()
ideal_score = topdata_df.groupby(['processingLevel.id'])['idealScore'].sum()

top_merged = pd.DataFrame()
top_merged['actualScore'] = actual_score
top_merged['idealScore'] = ideal_score

top_merged.plot(y=["actualScore", "idealScore"], use_index=True, kind="bar", title="Top 50/75 - Levels of Service")

# Try all the cloud hosted data

In [ ]:
all_cloud_data_df = pd.DataFrame()

for provider in provider_list:
    print(f'Retrieving collections for: {provider}')
    provider_df = query_cmr_graph_for_collections(
        {'cloudHosted': True, 'provider': provider}
    )
    print(f'Retrieved {len(provider_df)} collections\n')
    all_cloud_data_df = pd.concat([all_cloud_data_df, provider_df], ignore_index=True)

extract_all_first_native_data_formats(all_cloud_data_df)
score_all_collections(all_cloud_data_df)
display(all_cloud_data_df)

# Visually display level of service compliance for all cloud-hosted collections:

In [ ]:
actual_score = all_cloud_data_df.groupby(['processingLevel.id'])['actualScore'].sum()
ideal_score = all_cloud_data_df.groupby(['processingLevel.id'])['idealScore'].sum()

merged = pd.DataFrame()
merged['actualScore'] = actual_score
merged['idealScore'] = ideal_score

merged.plot(y=["actualScore", "idealScore"], use_index=True, kind="bar", title="All Cloud Data - Levels of Service")

# Identify areas of opportunity

Looking for biggest areas to focus on within the Top 50/75 data:

In [ ]:
opportunities = all_cloud_data_df.groupby(['processingLevel.id','nativeDataFormat1']).size()
opportunities = opportunities.unstack()
opportunities_t = opportunities.T
plt.figure(figsize=(10,40))
sns.heatmap(opportunities_t, vmin=10, vmax=200, cmap="coolwarm", annot=True)


# Service Gurus - phone a friend

Looking for gurus to talk to (DAACs who have wired up at least 1 service in their cloud data)

In [ ]:
service_gurus = all_cloud_data_df.loc[all_cloud_data_df['services.count'] >= 1]
display(service_gurus.groupby(['provider']).size())

# While at it, make our other metric generation super easy too

In [ ]:
collectionCounts = all_cloud_data_df.groupby(['provider']).size()
granuleCounts = all_cloud_data_df.groupby(['provider'])['granules.count'].sum()

merged = pd.DataFrame()
merged['granuleCount'] = granuleCounts
merged['collectionCount']= collectionCounts

display(merged)

merged.plot(y=["granuleCount"], use_index=True, kind="bar", title="All Cloud Data - Granules per Provider")

# Things we learned:
* 2 of the top 75 have 2 entries with the same shortname (GLDAS product v2.0 and v2.1). Version 2.1 has been used in this analysis.
* Our metadata is still _really_ inconsistent.  Data formats entirely missing, processing levels not standardized.
* It'd be helpful to easily query on 'hasOPeNDAP' (Amy's working this PR)
* It'd be helpful to easily query on 'hasVariableSubetting' and 'hasReprojection'. There is a proxy in this analysis for now that combines whether a collection has variables with whether it is associated with a service that offers variable subsetting.
* Pandas makes a lot of things really, really easy.  But, the graph nature of our metadata means that we hit limitations with just using panda.
* PO.DAAC's recent increase in cloud collections made pagination of CMR-graph requests a necessity.

# Bonus: Querying for all services:

In [ ]:
def query_cmr_graph_for_services():
    """ This query will retrieve information on all services, including
        which collections are configured for each.

        This query does not currently use pagination as there are currently
        only a limited number (< 100) of service records in production.

    """
    graphql_rooturl = "https://graphql.earthdata.nasa.gov/api"

    services_query = """
    query {
      services(limit: 2000) {
        items {
          type
          conceptId
          name
          serviceOptions
          supportedInputProjections
          supportedOutputProjections
          supportedReformattings
          collections {
            count
            items {
              conceptId
              cloudHosted
            }
          }
        }
      }
    }
    """

    response = requests.post(url=graphql_rooturl, json={"query": services_query})

    if response.status_code != 200:
        print("response status code: ", response.status_code)
        print("response : ", response.content)
        print("request: ", response.request)

    # Shove it into a dataframe
    json_data = json.loads(response.text)
    df_data = json_data['data']['services']['items']
    new_df = pd.json_normalize(df_data)

    return new_df
    

services_df = query_cmr_graph_for_services()
display(services_df)